In [2]:
!pip install -q tensorflow

In [6]:
# 1. Подключение Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Папки с данными
train_dir = '/content/dataset/train'
val_dir = '/content/dataset/val'

# 3. Импорты
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers

# 4. Параметры
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS_TOP = 5        # Сначала обучаем только верхушку
EPOCHS_FINE_TUNE = 20 # Потом дообучаем всю сеть

# 5. Загрузчики данных
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# 6. Модель
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base_model.trainable = False  # сначала замораживаем

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# 7. Компиляция и обучение только верхушки
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Обучаем только верхние слои...")
history_top = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_TOP
)

# 8. Разморозка EfficientNet
print("Размораживаем всю модель...")
base_model.trainable = True

# Обычно уменьшают скорость обучения после разморозки
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 9. Дообучение всей модели
print("Дообучаем всю модель...")
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_FINE_TUNE
)

# 10. Сохранение
model.save('/content/drive/MyDrive/weed_model_efficientnet.keras')
print("✅ Модель полностью обучена и сохранена!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 1461 images belonging to 5 classes.
Found 755 images belonging to 5 classes.
Обучаем только верхние слои...
Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 49s 291ms/step - accuracy: 0.2013 - loss: 1.6285 - val_accuracy: 0.3245 - val_loss: 1.5102
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.2544 - loss: 1.5749 - val_accuracy: 0.3245 - val_loss: 1.5235
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 6s 61ms/step - accuracy: 0.2459 - loss: 1.5733 - val_accuracy: 0.3245 - val_loss: 1.5148
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - accuracy: 0.2726 - loss: 1.5748 - val_accuracy: 0.3245 - val_loss: 1.5301
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 67ms/step - accuracy: 0.2474 - loss: 1.5778 - val_accuracy: 0.3245 - val_loss: 1.5147
Размораживаем всю модель...
Дообучаем всю модель...
Epoch 1/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 148s 740ms/step - accuracy: 0.273